First, let's set up all of our credentials for querying data from CJA. We're going to do this in two different ways - via the reporting API, and via the new PostgreSQL connector. Let's start with the reporting API, handled via cjapy courtesy of Julien Piccini.

In [ ]:
# Thanks to Julien Piccini for his hard work on the cjapy library!
import cjapy
'''
    For using cjapy, you'll need a config file that contains the following:
    {
        "org_id": "YOURORGID@AdobeOrg",
        "client_id": "YOURCLIENTID",
        "secret": "YOURCLIENTSECRET",
        "tech_id": "YOURTECHID@techacct.adobe.com",
        "pathToKey":"path_to_private.key"
    }
'''

# Instantiate cjapy
cjapy.importConfigFile("python_config.json")

cja = cjapy.CJA()

Check to see which Data Views you have available and grab the id of the Data View you want.

In [ ]:
# Get a list of all the Data Views available
data_views = cja.getDataViews()

# Extract and print the name and ID
for index, row in data_views.iterrows():
    print(f"Data View Name: {row['name']}, Data View ID: {row['id']}")

# Set the Data View id we want into a variable
data_view = "dv_62ba17d5a5d7845496f5fb4d"

You can also see which metrics and dimensions you have to work with using the getDimensions and getMetrics functions

In [ ]:
# Grab all available metrics
metrics = cja.getMetrics(data_view)

# Extract and print the metric name and ID
for index, row in metrics.iterrows():
    print(f"Metric: {row['name']}, ID: {row['id']}")

# Grab all available dimensions
dimensions = cja.getDimensions(data_view)

# Extract and print the dimension name and ID
for index, row in dimensions.iterrows():
    print(f"Dimension: {row['name']}, ID: {row['id']}")

Now let's pull a report for orders by product name

In [ ]:

# Pick dimension and metric
dimension = "variables/product_name"
metric = "metrics/orders"
dateRange = "2024-01-01T00:00:00.000/2024-01-17T00:00:00.000"

# Define the report request
myRequest = cjapy.RequestCreator()
myRequest.setDataViewId(data_view)
myRequest.setDimension(dimension)
myRequest.addMetric(metric)
myRequest.addGlobalFilter(dateRange)

# Pull and print the report from CJA
myReport = cja.getReport(myRequest)
myReport.dataframe[[dimension, metric]]

You can also pull the same data via the new BI connector for CJA using SQL! Some may find this approach easier if you're more familiar with SQL. Either way, the results you get will be exactly the same and hit CJA in the same way. Here's how you can do that:

In [ ]:
from sqlalchemy import create_engine
import pandas as pd

# Create a connection to the PostgreSQL database
username = '8F99160E571FC0427F000101@AdobeOrg'
password = 'eNqrVkrMSVeyUgoKNjI1U9JRqjAtBfIyc4vj8xINdbNTK3UTS3QN9ZJTi4CS2ZkpWCSBEpklJUCJxBKlWgC4CRff.eNplUl1v4jAQ_C95ToId8ol0UlOgKrpCoYXSIKRosR1qGuIocShQ9b-fTeg9tH5I7FnvenZnPg1OjZ6BA4Sx67mo2w3DNPBdAt3Qt6iDqeX6GFtAmG9FbgDMQ8yFDUqbD8cwDXkqmcoHQlhdp1K8s0KhJOeskOmlNDuSFMpSoU3NqhYLvdDHw8j3u7jv4rsYxW7kDW_jm4A6Ac68SAUIIuAomBDHZiq7liD1U59ro1ZvcVGsjd7aeJOyrHudDt_XVgHYBio2LBdbXthE7DXcuV7vHHAnOcfueP7GJ8uhXM3j04ojtFqO0MM83yXLRE7mC5zsyOlxMHIm59X7wzPuz855_2kwPo53C_Q4yIdPu_g0Hg7R7PzuzuarOFk-HZL9okmcSK6NL0UVasXzSkgfoe36th_dDrrY8dwwGtwhR3Xdj8LYv7lw1nT16GRp9JBpZFuV8RpMR9PRfPU6nQbj-8Xf-_HsxXmNYy_5c1l6LK1-fuSgbhRgFEReSoBsHJ85Fs2ySOsXWsBIZCEgAWNh6GZhdtVvL7hK7wZeuAFPj7ncgALGjD4zMhHDF_NBfKitirBjyStWp7zQCvou0kszrphShqYgfxpJ0yOiNcjG3kMBW2Yqq4hGmYPkTS1ZZatsagKlXCqNIFflM_HzbO_EJs2agmjsV7CsxI4RRWFaCdoQ2ReFZEf5614lclabsR72aGBehs6FrbxZsS2vZXX65nIJKNNyExqqnExYy726YJd--YGpDkRDzX1Zm6rHglNTVNu2hP6k6gQFP4OmULdQSb7_NpDy_57uIZVQSSjMq1eNr3_uhysE.eNoBAAH__hDND82YXjHcBH7ZX4Tw9zcKAgr-oO_kpzvY5GV5CxeA0caRbd9TlgzlRbsjz0wwmMuCA8RQLhQytEmNKDH-4AmOhg3CgIxjwu9aVVJ1ouGTx6OwVRy335--crkWQRCnTULbGKxe5VhxIxsFOJ7awBmSUBoHBjyh2JygNCk2oI8c1lDfdScw_7r5yb5LyX9dSAa6ur94U3ssaL9qWA1Upn0GImJ44Xw881hHU4YjVSqE8xV2ZX1QHc1FhMXP8lbgsGjWEIVfRYOmCBCsNjvj5B6HBR-eLe-0soYOCQfnZDYsWQnbRMQuqabZwIgvibvIQx1B6AvTQsYecMZA2gfDWb36OHfe'
host = 'amc.platform-query.adobe.io'
port = 80
dbname = 'prod:cja'
engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{dbname}')

# Grab the list of Data Views available
query = """
    SELECT tablename FROM pg_catalog.pg_tables WHERE schemaname != 'pg_catalog' AND schemaname != 'information_schema';
"""
df = pd.read_sql(query, engine)
print(df)

# Query the dataview and read the results
query = """
    SELECT product_name, sum(orders) as orders
    FROM dv_adobe_store
    GROUP BY product_name
    ORDER BY orders DESC
"""
df = pd.read_sql(query, engine)
df